In [1]:
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/research'

In [2]:
import os

os.chdir('../')
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf'

### Data Ingestion Configuration
Organize and store configuration settings for the data ingestion process.

#### Configuration entities

In [3]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

#### Configuration manager

In [4]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utilities import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        root_dir = Path(config.root_dir)
        local_data_file = Path(config.local_data_file)
        unzip_dir = Path(config.unzip_dir)

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=root_dir,
            source_URL=config.source_URL,
            local_data_file=local_data_file,
            unzip_dir=unzip_dir
        )

        return data_ingestion_config

#### Data Ingestion Component

In [6]:
from urllib import request
import zipfile
from cnn_classifier import logger
from cnn_classifier.utils.utilities import get_size
import re
import shutil

In [70]:
%pwd

'/Users/neehanth/Documents/Data Scientist'

In [ ]:
# # Check the labels
# images_dir = Path("artifacts/data_ingestion/data/images")

# for f in images_dir.iterdir():
#     print(f.name.split('.')[0], " | ", f.suffix.lower())

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """Initialize DataIngestion with the given configuration."""
        self.config = config
    
    def download_data(self):
        """
        Download the data file from the source URL if it does not already exist locally.

        Returns:
            str: Path to the downloaded (or existing) local data file.

        Raises:
            Exception: If an error occurs during download.
        """
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"Downloaded {filename} with following info:\n{headers}")
            else:
                logger.info(f"Data already exists at {self.config.local_data_file} ({get_size(Path(self.config.local_data_file))}), skipping download")
            
            return self.config.local_data_file
        except Exception as e:
            logger.error(f"Error downloading data: {e}")
            raise
    
    def extract_zip_file(self):
        """Extract the contents of the downloaded zip file into the configured directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file_ref:
            zip_file_ref.extractall(unzip_path)
        
        # Rename top-level folder: train/ or Train/ -> images/
        src = None
        for candidate in [unzip_path / "train", unzip_path / "Train"]:
            if candidate.exists() and candidate.is_dir():
                src = candidate
                break
        if src is None:
            return
        
        dst = unzip_path / "images"
        
        # If already renamed in a previous run, do nothing
        if dst.exists() and dst.is_dir():
            return
        
        # Avoid accidental overwrite
        if dst.exists():
            raise FileExistsError(f"Cannot rename to '{dst}': path exists.")
        
        src.rename(dst)
    
    def organize_images_into_class_folders(self) -> None:
        """
        Moves images from artifacts/.../data/images/*.jpg into class label folders.
        Label is inferred from filename prefix (before first dot).
        """
        self.images_dir = self.config.unzip_dir / "images"
        if not self.images_dir.exists():
            raise FileNotFoundError(f"images folder not found at: {self.images_dir}")
        
        class_dirs = {
            "healthy": self.images_dir / "Healthy",
            "salmo": self.images_dir / "Salmonella",
            "cocci": self.images_dir / "Coccidiosis",
            "ncd": self.images_dir / "New Castle Disease",
        }
        
        # Ensure folders exist
        for cls in class_dirs.values():
            cls.mkdir(parents=True, exist_ok=True)
        
        def normalize_key(stem: str) -> str:
            s = stem.lower().strip()
            s = re.sub(r"[^a-z]", "", s)
            if s.startswith("pcr"):
                s = s[3:]
            return s
        
        moved, skipped, unknown = 0, 0, 0
        
        # Move images directly under images/
        for f in self.images_dir.iterdir():
            if not f.is_file():
                continue
            if f.suffix.lower() not in {".jpg", ".jpeg", ".png"}:
                continue

            # filename like: pcrhealthy.123.jpg -> image filename prefix is "pcrhealthy"
            img_filename_prefix = f.name.split(".")[0]
            key = normalize_key(img_filename_prefix)

            target_dir = class_dirs.get(key)
            if target_dir is None:
                unknown += 1
                continue
            
            dest = target_dir / f.name
            if dest.exists():
                skipped += 1
                continue

            shutil.move(str(f), str(dest))
            moved += 1
        
        logger.info(f"Organize images: moved={moved}, skipped={skipped}, unknown={unknown}")
    
    def print_data_summary(self, expected_total: int = 8067):
        img_exts = {".jpg", ".jpeg", ".png"}

        logger.info(f"Data Summary @ {self.images_dir}")
        logger.info("Classes: " + ", ".join(sorted([f.name for f in self.images_dir.iterdir() if f.is_dir()])))

        class_counts = {}
        total = 0

        for class_dir in sorted([f for f in self.images_dir.iterdir() if f.is_dir()]):
            n = sum(1 for f in class_dir.rglob("*") if f.suffix.lower() in img_exts)
            class_counts[class_dir.name] = n
            total += n
        
        logger.info("Data Summary")
        for k, v in class_counts.items():
            logger.info(f"{k:20s} {v}")

        logger.info(f"\nTOTAL: {total}")

#### Data Ingesiton Pipeline

In [13]:
# Create the data ingestion pipeline
try:
    config = ConfigurationManager()
    data_ingestion_cofig = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_cofig)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    data_ingestion.organize_images_into_class_folders()
    data_ingestion.print_data_summary()
except Exception as e:
    raise e

[2025-12-21 15:21:28,680]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-21 15:21:28,682]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-21 15:21:28,683]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts
[2025-12-21 15:21:28,684]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/data_ingestion
[2025-12-21 15:21:35,320]: cnnClassiferLogger - INFO - 1911480522 - Downloaded artifacts/data_ingestion/data.zip with following info:
Connection: close
Content-Length: 278791023
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "6641a21b18310e23c1b848c99c0552b66eaf58bb3821f52563aecec448e0704e"
Last-Modified: Sun, 21 Dec 2025 21:20:58 GMT
Strict-Transport-Security: max-age=31557600
timing-allo

In [10]:
# Verify the image count
TOTAL_IMAGES = 8067
IMAGES_DIR = Path("artifacts/data_ingestion/data/images")

class_counts = {}
total = 0

for class_dir in sorted([p for p in IMAGES_DIR.iterdir() if p.is_dir()]):
    n = sum(1 for f in class_dir.rglob("*") if f.suffix.lower() in {".jpg", ".jpeg", ".png"})
    class_counts[class_dir.name] = n
    total += n

for k, v in class_counts.items():
    print(f"{k:20s} {v}")

print("\nTOTAL:", total)
print("EXPECTED:", TOTAL_IMAGES)

Coccidiosis          2476
Healthy              2404
New Castle Disease   562
Salmonella           2625

TOTAL: 8067
EXPECTED: 8067
